## This is a note experimenting on the inconsistency issue of LIME and how BayLIME (with 3 options) improves on it 

In [ ]:
import sys
sys.path.append("..")# allow the notebook to find the parent folder
from sklearn.datasets import load_boston,load_breast_cancer
import sklearn.ensemble
import sklearn.linear_model
import sklearn.model_selection
import numpy as np
from sklearn.metrics import r2_score
np.random.seed(1)
import lime
import lime.lime_tabular
from lime import submodular_pick
from lime import calculate_posteriors
import csv
import math


#load example dataset
boston = load_boston()
#cancer= load_breast_cancer()

#print a description of the variables
print(boston.DESCR)
#print(cancer.DESCR)

#train a regressor
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=1000)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(boston.data, boston.target, train_size=0.80, test_size=0.20)
rf.fit(train, labels_train);

#train a classifier
# rf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000)
# train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(cancer.data, cancer.target, train_size=0.80, test_size=0.20)
# rf.fit(train, labels_train);






In [ ]:
# generate an "explainer" object
categorical_features  = np.argwhere(np.array([len(set(boston.data[:,x])) for x in range(boston.data.shape[1])]) <= 10).flatten()


explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=boston.feature_names, 
                                                   class_names=['price'], categorical_features=categorical_features, 
                                                    verbose=False, mode='regression',
                                                   discretize_continuous=False,feature_selection='none',
                                                   sample_around_instance=False)

# An example of classifier, rather than regressor...
# categorical_features = np.argwhere(np.array([len(set(cancer.data[:,x])) for x in range(cancer.data.shape[1])]) <= 10).flatten()

# explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=cancer.feature_names, 
#                                                    categorical_features=categorical_features, 
#                                                    class_names=cancer.target_names,
#                                                    verbose=False, mode='classification',
#                                                    discretize_continuous=False,
#                                                    feature_selection='none')


#generate an explanation for testing..
instance = 3
#use rf.predict_proba for classfication 
exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_non_info_prior',
                                 num_samples=100,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior'

exp.show_in_notebook(show_table=True)
#fig = exp.as_pyplot_figure(label=1)



print(exp.as_list())
#fig = exp.as_pyplot_figure(label=2)

In [ ]:
exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                  model_regressor='BayesianRidge_inf_prior_fit_alpha',
                                  num_samples=100,
                                  #labels=labels_test[i],
                                  top_labels=1)
#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior', 'BayesianRidge_inf_prior_fit_alpha'
  

alpha_init=1
lambda_init=1
with open('.\posterior_configure.csv') as csv_file:
    csv_reader=csv.reader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 1:
            alpha_init=float(row[0])
            lambda_init=float(row[1])
        line_count=line_count+1

exp.show_in_notebook(show_table=True)

exp=calculate_posteriors.get_posterior(exp,'.\data\prior_knowledge_tabular.csv',hyper_para_alpha=alpha_init, hyper_para_lambda=lambda_init,
                                        label=1)

exp.show_in_notebook(show_table=True)

In [ ]:
exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                  model_regressor='Bay_info_prior',
                                  num_samples=100,
                                  #labels=labels_test[i],
                                  top_labels=1)
#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior', 'BayesianRidge_inf_prior_fit_alpha'
  

alpha_init=1
lambda_init=1
with open('.\posterior_configure.csv') as csv_file:
    csv_reader=csv.reader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 1:
            alpha_init=float(row[0])
            lambda_init=float(row[1])
        line_count=line_count+1

exp.show_in_notebook(show_table=True)

exp=calculate_posteriors.get_posterior(exp,'.\data\prior_knowledge_tabular.csv',hyper_para_alpha=alpha_init, hyper_para_lambda=lambda_init,
                                        label=1)

exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
k=3#number of explanations 
m=13# number of features
i=1
instance=3
explanations=np.array([])
while i<=k:
    exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                 model_regressor='Bay_non_info_prior',
                                 num_samples=100,
                                 #labels=labels_test[i],
                                 )#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'
    temp_list=exp.as_list()
    temp_array = np.array(temp_list)
    explanations=np.append(explanations,temp_array)
    i=i+1

In [ ]:
exps=explanations.reshape(k,2*m)# k exps, 13 features for this instance.. 

In [ ]:
for exp in exps:
    #print(exp)
    i=1
    temp_vector=np.array([])
    while i<=(2*m-1):
        temp_vector=np.append(temp_vector,float(exp[i]))
        i=i+2
    #print(temp_vector)
    normlised_temp_vector=temp_vector/np.linalg.norm(temp_vector)
    #print(normlised_temp_vector)
    i=1
    while i<=(2*m-1):
        exp[i]=normlised_temp_vector[math.floor(i/2)]
        i=i+2

In [ ]:
print(exps)

In [ ]:
feature_names=np.array([])
i=0
while i<=(2*m-1):
    feature_names=np.append(feature_names,exps[0,i])
    i=i+2
print(feature_names)

In [ ]:
def rankings_in_k_exp (feature, k_exps):
    ranks=np.array([])
    for exp in k_exps:
        rank=math.ceil(exp.tolist().index(feature)/2)+1
        ranks=np.append(ranks,rank)
    return ranks

print(rankings_in_k_exp('LSTAT',exps))

In [ ]:
def importance_in_k_exp (feature, k_exps):
    importance_s=np.array([])
    for exp in k_exps:
        importance=exp[exp.tolist().index(feature)+1]
        importance_s=np.append(importance_s,importance)
    return importance_s

print(importance_in_k_exp('INDUS',exps))

In [ ]:
g_i_s=np.array([])
f_i_s=np.array([])
for feature in feature_names:
    g_i=importance_in_k_exp(feature,exps)
    f_i=rankings_in_k_exp(feature,exps)
    g_i_s=np.append(g_i_s,g_i)
    f_i_s=np.append(f_i_s,f_i)
g_i_s=g_i_s.reshape(m,k)
f_i_s=f_i_s.reshape(m,k)
print(g_i_s)
print(f_i_s)

In [ ]:
#now calculate the index of dispersion of ranks for each feature
IoD_f_i_s=np.array([])
for f_i in f_i_s:
    if np.mean(f_i)==0:
        IoD_f_i=0
        IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
    else:
        IoD_f_i=np.var(f_i)/np.mean(f_i)
        IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
print(IoD_f_i_s)

In [ ]:
#now calculate the weighted importance for each feature
weights_g_i_s=np.array([])
for g_i in g_i_s:
    weight=np.mean(abs(g_i.astype(np.float)))
    weights_g_i_s=np.append(weights_g_i_s,weight)
weights_g_i_s=weights_g_i_s/sum(weights_g_i_s)
print(weights_g_i_s)

In [ ]:
np.dot(weights_g_i_s,IoD_f_i_s)

In [ ]:
def kendall_w(expt_ratings):
    if expt_ratings.ndim!=2:
        raise 'ratings matrix must be 2-dimensional'
    m = expt_ratings.shape[0] #raters
    n = expt_ratings.shape[1] # items rated
    denom = m**2*(n**3-n)
    rating_sums = np.sum(expt_ratings, axis=0)
    S = n*np.var(rating_sums)
    return 12*S/denom

In [ ]:
ken_w=kendall_w(f_i_s.T)
print(ken_w)

## Now we have shown a toy example of calculating the inconsistency of k explanations for a given number of samples, while we want to see inconsistency as a function of the number of samples... Let us do it now...

In [ ]:
n=10 #number of samples
inconsistency_non_info=np.array([])
ken_w_non_info=np.array([])
while n<=2000:


    k=100#number of explanations 
    m=13# number of features
    i=1
    instance=3
    explanations=np.array([])
    while i<=k:
        exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                         model_regressor='Bay_non_info_prior',
                                         num_samples=n)#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'
        temp_list=exp.as_list()
        temp_array = np.array(temp_list)
        explanations=np.append(explanations,temp_array)
        i=i+1
    exps=explanations.reshape(k,2*m)# k exps, 13 features for this instance.. 
    for exp in exps:
    #print(exp)
        i=1
        temp_vector=np.array([])
        while i<=(2*m-1):
            temp_vector=np.append(temp_vector,float(exp[i]))
            i=i+2
    #print(temp_vector)
        normlised_temp_vector=temp_vector/np.linalg.norm(temp_vector)
    #print(normlised_temp_vector)
        i=1
        while i<=(2*m-1):
            exp[i]=normlised_temp_vector[math.floor(i/2)]
            i=i+2
    feature_names=np.array([])
    i=0
    while i<=(2*m-1):
        feature_names=np.append(feature_names,exps[0,i])
        i=i+2
    
    g_i_s=np.array([])
    f_i_s=np.array([])
    for feature in feature_names:
        g_i=importance_in_k_exp(feature,exps)
        f_i=rankings_in_k_exp(feature,exps)
        g_i_s=np.append(g_i_s,g_i)
        f_i_s=np.append(f_i_s,f_i)
    g_i_s=g_i_s.reshape(m,k)
    f_i_s=f_i_s.reshape(m,k)
    IoD_f_i_s=np.array([])
    for f_i in f_i_s:
        if np.mean(f_i)==0:
            IoD_f_i=0
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
        else:
            IoD_f_i=np.var(f_i)/np.mean(f_i)
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
    weights_g_i_s=np.array([])
    for g_i in g_i_s:
        weight=np.mean(abs(g_i.astype(np.float)))
        weights_g_i_s=np.append(weights_g_i_s,weight)
    weights_g_i_s=weights_g_i_s/sum(weights_g_i_s)
    
    inconsistency_non_info=np.append(inconsistency_non_info,np.dot(weights_g_i_s,IoD_f_i_s))
    
    ken_w_non_info=np.append(ken_w_non_info,kendall_w(f_i_s.T))
    
    print(n)
    n=n+10

In [ ]:
n=10 #number of samples
inconsistency_non_Bay=np.array([])
ken_w_non_Bay=np.array([])
while n<=2000:

    k=100#number of explanations 
    m=13# number of features
    i=1
    instance=3
    explanations=np.array([])
    while i<=k:
        exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                         model_regressor='non_Bay',
                                         num_samples=n)#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'
        temp_list=exp.as_list()
        temp_array = np.array(temp_list)
        explanations=np.append(explanations,temp_array)
        i=i+1
    exps=explanations.reshape(k,2*m)# k exps, 13 features for this instance.. 
    for exp in exps:
    #print(exp)
        i=1
        temp_vector=np.array([])
        while i<=(2*m-1):
            temp_vector=np.append(temp_vector,float(exp[i]))
            i=i+2
    #print(temp_vector)
        normlised_temp_vector=temp_vector/np.linalg.norm(temp_vector)
    #print(normlised_temp_vector)
        i=1
        while i<=(2*m-1):
            exp[i]=normlised_temp_vector[math.floor(i/2)]
            i=i+2
    feature_names=np.array([])
    i=0
    while i<=(2*m-1):
        feature_names=np.append(feature_names,exps[0,i])
        i=i+2
    
    g_i_s=np.array([])
    f_i_s=np.array([])
    for feature in feature_names:
        g_i=importance_in_k_exp(feature,exps)
        f_i=rankings_in_k_exp(feature,exps)
        g_i_s=np.append(g_i_s,g_i)
        f_i_s=np.append(f_i_s,f_i)
    g_i_s=g_i_s.reshape(m,k)
    f_i_s=f_i_s.reshape(m,k)
    IoD_f_i_s=np.array([])
    for f_i in f_i_s:
        if np.mean(f_i)==0:
            IoD_f_i=0
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
        else:
            IoD_f_i=np.var(f_i)/np.mean(f_i)
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
    weights_g_i_s=np.array([])
    for g_i in g_i_s:
        weight=np.mean(abs(g_i.astype(np.float)))
        weights_g_i_s=np.append(weights_g_i_s,weight)
    weights_g_i_s=weights_g_i_s/sum(weights_g_i_s)
    inconsistency_non_Bay=np.append(inconsistency_non_Bay,np.dot(weights_g_i_s,IoD_f_i_s))
    ken_w_non_Bay=np.append(ken_w_non_Bay,kendall_w(f_i_s.T))
    
    print(n)
    n=n+10

In [ ]:
n=10 #number of samples
inconsistency_info_prior_fit_alpha=np.array([])
ken_w_info_prior_fit_alpha=np.array([])
while n<=2000:

    k=100#number of explanations 
    m=13# number of features
    i=1
    instance=3
    explanations=np.array([])
    while i<=k:
        exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                         model_regressor='BayesianRidge_inf_prior_fit_alpha',
                                         num_samples=n)#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'
        
        alpha_init=1
        lambda_init=1
        with open('.\posterior_configure.csv') as csv_file:
            csv_reader=csv.reader(csv_file)
            line_count = 0
            for row in csv_reader:
                if line_count == 1:
                    alpha_init=float(row[0])
                    lambda_init=float(row[1])
                line_count=line_count+1

        exp=calculate_posteriors.get_posterior(exp,'.\data\prior_knowledge_tabular.csv',hyper_para_alpha=alpha_init, hyper_para_lambda=lambda_init,
                                        label=1)
        
        temp_list=exp.as_list()
        temp_array = np.array(temp_list)
        explanations=np.append(explanations,temp_array)
        i=i+1
    exps=explanations.reshape(k,2*m)# k exps, 13 features for this instance.. 
    for exp in exps:
    #print(exp)
        i=1
        temp_vector=np.array([])
        while i<=(2*m-1):
            temp_vector=np.append(temp_vector,float(exp[i]))
            i=i+2
    #print(temp_vector)
        normlised_temp_vector=temp_vector/np.linalg.norm(temp_vector)
    #print(normlised_temp_vector)
        i=1
        while i<=(2*m-1):
            exp[i]=normlised_temp_vector[math.floor(i/2)]
            i=i+2
    feature_names=np.array([])
    i=0
    while i<=(2*m-1):
        feature_names=np.append(feature_names,exps[0,i])
        i=i+2
    
    g_i_s=np.array([])
    f_i_s=np.array([])
    for feature in feature_names:
        g_i=importance_in_k_exp(feature,exps)
        f_i=rankings_in_k_exp(feature,exps)
        g_i_s=np.append(g_i_s,g_i)
        f_i_s=np.append(f_i_s,f_i)
    g_i_s=g_i_s.reshape(m,k)
    f_i_s=f_i_s.reshape(m,k)
    IoD_f_i_s=np.array([])
    for f_i in f_i_s:
        if np.mean(f_i)==0:
            IoD_f_i=0
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
        else:
            IoD_f_i=np.var(f_i)/np.mean(f_i)
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
    weights_g_i_s=np.array([])
    for g_i in g_i_s:
        weight=np.mean(abs(g_i.astype(np.float)))
        weights_g_i_s=np.append(weights_g_i_s,weight)
    weights_g_i_s=weights_g_i_s/sum(weights_g_i_s)
    inconsistency_info_prior_fit_alpha=np.append(inconsistency_info_prior_fit_alpha,np.dot(weights_g_i_s,IoD_f_i_s))
    
    ken_w_info_prior_fit_alpha=np.append(ken_w_info_prior_fit_alpha,kendall_w(f_i_s.T))
    
    print(n)
    n=n+10

In [ ]:
n=10 #number of samples
inconsistency_full_info_prior=np.array([])
ken_w_full_info_prior=np.array([])

while n<=2000:

    k=100#number of explanations 
    m=13# number of features
    i=1
    instance=3
    explanations=np.array([])
    while i<=k:
        exp = explainer.explain_instance(test[instance], rf.predict,#num_features=13,
                                         model_regressor='Bay_info_prior',
                                         num_samples=n)#'non_Bay' 'Bay_non_info_prior' 'Bay_info_prior' 'BayesianRidge_inf_prior_fit_alpha'
        
        
        alpha_init=1
        lambda_init=1
        with open('.\posterior_configure.csv') as csv_file:
            csv_reader=csv.reader(csv_file)
            line_count = 0
            for row in csv_reader:
                if line_count == 1:
                    alpha_init=float(row[0])
                    lambda_init=float(row[1])
                line_count=line_count+1

        exp=calculate_posteriors.get_posterior(exp,'.\data\prior_knowledge_tabular.csv',hyper_para_alpha=alpha_init, hyper_para_lambda=lambda_init,
                                        label=1)
        
        
        temp_list=exp.as_list()
        temp_array = np.array(temp_list)
        explanations=np.append(explanations,temp_array)
        i=i+1
    exps=explanations.reshape(k,2*m)# k exps, 13 features for this instance.. 
    
    for exp in exps:
    #print(exp)
        i=1
        temp_vector=np.array([])
        while i<=(2*m-1):
            temp_vector=np.append(temp_vector,float(exp[i]))
            i=i+2
    #print(temp_vector)
        normlised_temp_vector=temp_vector/np.linalg.norm(temp_vector)
    #print(normlised_temp_vector)
        i=1
        while i<=(2*m-1):
            exp[i]=normlised_temp_vector[math.floor(i/2)]
            i=i+2
    feature_names=np.array([])
    i=0
    while i<=(2*m-1):
        feature_names=np.append(feature_names,exps[0,i])
        i=i+2
    
    g_i_s=np.array([])
    f_i_s=np.array([])
    for feature in feature_names:
        g_i=importance_in_k_exp(feature,exps)
        f_i=rankings_in_k_exp(feature,exps)
        g_i_s=np.append(g_i_s,g_i)
        f_i_s=np.append(f_i_s,f_i)
    g_i_s=g_i_s.reshape(m,k)
    f_i_s=f_i_s.reshape(m,k)
    IoD_f_i_s=np.array([])
    for f_i in f_i_s:
        if np.mean(f_i)==0:
            IoD_f_i=0
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
        else:
            IoD_f_i=np.var(f_i)/np.mean(f_i)
            IoD_f_i_s=np.append(IoD_f_i_s,IoD_f_i)
    weights_g_i_s=np.array([])
    for g_i in g_i_s:
        weight=np.mean(abs(g_i.astype(np.float)))
        weights_g_i_s=np.append(weights_g_i_s,weight)
    weights_g_i_s=weights_g_i_s/sum(weights_g_i_s)
    inconsistency_full_info_prior=np.append(inconsistency_full_info_prior,np.dot(weights_g_i_s,IoD_f_i_s))
    
    ken_w_full_info_prior=np.append(ken_w_full_info_prior,kendall_w(f_i_s.T))
    
    print(n)
    n=n+10

In [ ]:
import matplotlib.pyplot as plt
x_index=np.array([])
i=10
while i<=2000:
    x_index=np.append(x_index,i)
    i=i+10
    

plt.plot(x_index,inconsistency_non_Bay,linestyle='-',color='red',label='LIME')
plt.plot(x_index,inconsistency_non_info,linestyle='-',color='green',label='BayLIME with non-informative priors')
plt.plot(x_index,inconsistency_info_prior_fit_alpha,linestyle='-',color='blue',label=r'BayLIME with partial informative priors ($\lambda=20$)')
plt.plot(x_index,inconsistency_full_info_prior,linestyle='-',color='orange',label=r'BayLIME with full informative priors ($\alpha=1,\lambda=20$)')

#plt.xscale('log')
plt.legend(loc='upper right',fontsize=10)#bbox_to_anchor=(1,1)
#plt.axis([10, 300, 0,10])# set the ranges of axis
plt.xlabel('$n$')
plt.ylabel('inconsistency')
plt.grid(True)

plt.show()

In [ ]:
plt.plot(x_index,ken_w_non_Bay,linestyle='-',color='red',label='LIME')
plt.plot(x_index,ken_w_non_info,linestyle='-',color='green',label='BayLIME with non-informative priors')
plt.plot(x_index,ken_w_info_prior_fit_alpha,linestyle='-',color='blue',label=r'BayLIME with partial informative priors ($\lambda=20$)')
plt.plot(x_index,ken_w_full_info_prior,linestyle='-',color='orange',label=r'BayLIME with full informative priors ($\alpha=1,\lambda=20$)')

#plt.xscale('log')
plt.legend(fontsize=10)#bbox_to_anchor=(1,1) ,loc='upper right'
#plt.axis([10, 300, 0,10])# set the ranges of axis
plt.xlabel('$n$')
plt.ylabel('Kendall’s W')
plt.grid(True)

plt.show()